# Retrieve data from external sources
## Get structures from the COD and ICSD
Time: 2 mins

In [1]:
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.tools.dbimporters import DbImporterFactory
from ase.visualize import view
import spglib

## Import a structure from the COD database (http://crystallography.net/cod/)

#### Load the COD importer class and instantiate it

In [2]:
importer_class = DbImporterFactory('cod')
importer = importer_class()
importer

#### Query the COD for a given database formula

In [ ]:
cod_id = '1510230' # AuMg
# cod_id = '9009138' # CdI2
# cod_id = '9008845' # GaAs

In [3]:
query_results = importer.query(formula='Au Mg')

In [4]:
print len(query_results)

2


#### Get the cif file corresponding to the entry, and convert it to an AiiDA structure

In [5]:
entry = query_results.at(0)
cif = entry.get_cif_node()
print cif.get_formulae()

['Au Mg']


In [6]:
structure = cif._get_aiida_structure(converter='pymatgen')
print structure.get_formula()
print structure.get_ase()

AuMg
Atoms(symbols='MgAu', pbc=[True, True, True], cell=[[3.259, 0.0, 1.995561959210612e-16], [5.240871715296158e-16, 3.259, 1.995561959210612e-16], [0.0, 0.0, 3.259]], masses=..., positions=...)


#### Get the spacegroup of the structure with spglib

In [7]:
print spglib.get_spacegroup(structure.get_ase())

Pm-3m (221)


#### Store the structure and prints its pk

In [8]:
structure.store()
structure.pk

5223

In [ ]:
view(structure.get_ase())

## Import a structure from the ICSD database (https://www.fiz-karlsruhe.de/icsd.html)

#### Connect (with ssh tunnelling) to the server hosting the database

In [9]:
!ssh -fN -L 3306:localhost:3306 -L 8010:localhost:80 aiida@theossrv2.epfl.ch > /dev/null 2>&1

#### Load the ICSD importer class and instantiate it

In [10]:
importer_class = DbImporterFactory('icsd')
importer_parameters = {'server': 'http://theossrv2.epfl.ch/',
                   'host': '127.0.0.1',
                   'db': 'icsd',
                   'passwd': 'sql'
                  }
importer = importer_class(**importer_parameters)
importer

#### Query the ICSD for a given database entry

In [11]:
icsd_id = '617290' # graphite

In [12]:
query_results = importer.query(id=icsd_id)

In [13]:
print len(query_results)

1


#### Get the cif file corresponding to the entry, and convert it to an AiiDA structure

In [14]:
entry = query_results.at(0)
cif = entry.get_cif_node()
print cif.get_formulae()

['C1']


In [15]:
structure = cif._get_aiida_structure(converter='pymatgen')
print structure.get_formula()
print structure.get_ase()

C4
Atoms(symbols='C4', pbc=[True, True, True], cell=[[2.47, 0.0, 1.5124387969469814e-16], [-1.2349999999999994, 2.139082747347564, 1.5124387969469814e-16], [0.0, 0.0, 6.93]], masses=..., positions=...)


#### Get the spacegroup of the structure with spglib

In [16]:
print spglib.get_spacegroup(structure.get_ase())

Cmcm (63)


#### Store the structure and prints its pk

In [17]:
structure.store()
structure.pk

5224

#### Now access the terminal and type the following to visualise the structure
`verdi data structure show --format xcrysden 5084`